In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Tom Jones,Help Yourself,Love is like candy on a shelf \r\nYou want to...
1,R. Kelly,Leave Wit Me,[Chorus: (R Kelly)] \r\nBaby dont chu wanna l...
2,U2,Pop Muzik,"Muzik \r\nPop, pop pop, muzik \r\n \r\nPop ..."
3,Venom,Clarisse,Clarisse what perfume you wearing today no don...
4,Black Sabbath,Selling My Soul,Man of madness who lives in my head \r\nKeepi...
5,Andrea Bocelli,Rodrigo (In English),"Arunjuez, a place of love and dreams, \r\nWhe..."
6,Heart,Treat Me Well,Well I have lost some time but I have won \r\...
7,Overkill,Ironbound,"One by one, doing the crime \r\nNever thought..."
8,Pharrell Williams,Take It Off (Dim The Lights),We've just been laughin' \r\nChattin' but not...
9,You Am I,Ain't Gone And Open,Wide eyes show white \r\nWhite enough to shoo...


In [9]:
df['text'][0]

"Love is like candy on a shelf  \r\nYou want to taste and help yourself  \r\nThe sweetest things are there for you  \r\nHelp yourself take a few  \r\nThat's what I want you to do  \r\n  \r\nWe're always told repeatedly  \r\nThe very best in life is free  \r\nAnd if you want to prove it's true  \r\nBaby, I'm telling you this is what you should do  \r\n  \r\nJust help yourself to my lips to my arms  \r\nJust say the word and they're yours  \r\nJust help yourself to the love in my heart  \r\nYour smile has opened up the door  \r\nThe greatest wealth that exists in the world  \r\nCould never buy what I can give  \r\nJust help yourself to my lips to my arms  \r\nAnd then let's really start to live  \r\n  \r\nMy heart has love enough for two  \r\nMore than enough for me and you  \r\nI'm rich with love, a millionaire  \r\nI've so much it's unfair  \r\nWhy don't you take a share  \r\n  \r\nJust help yourself to my lips to my arms  \r\nJust say the word and they're yours  \r\nJust help yourself

In [10]:
# df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [14]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [17]:
similarity[0]

array([1.00000000e+00, 7.92966368e-03, 2.83713754e-03, ...,
       6.22999576e-02, 2.34949789e-04, 2.41636821e-02])

In [18]:
df[df['song'] == 'Crying Over You']

,artist,song,text
1341,UB40,Crying Over You,cri over you in the morn cri over you in the e...


In [23]:
def recommender(song_name):
    idx = df[df['song'] == song_name].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    songs = []
    for s_id in distances[1:5]:
        songs.append(df.iloc[s_id[0]].song)
        
    return songs

In [24]:
recommender("In Your Eyes")

['Layla', 'Jump', 'In The Country', 'September']

In [25]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))